<a href="https://colab.research.google.com/github/kavyaperavali/kavya_INFO-5731_spring_2020/blob/master/kavya_Copy_of_INFO5731_Assignment_four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Four**

In this assignment, you are required to conduct topic modeling, sentiment analysis based on **the dataset you created from assignment three**.

# **Question 1: Topic Modeling**

(30 points). This question is designed to help you develop a feel for the way topic modeling works, the connection to the human meanings of documents. Based on the dataset from assignment three, write a python program to **identify the top 10 topics in the dataset**. Before answering this question, please review the materials in lesson 8, especially the code for LDA and LSA. The following information should be reported:

(1) Features (top n-gram phrases) used for topic modeling.

(2) Top 10 clusters for topic modeling.

(3) Summarize and describe the topic for each cluster. 


In [0]:
# Write your code here
import nltk
nltk.download()


In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import csv
from nltk.tokenize import RegexpTokenizer

In [73]:
#Importing the dataset and converting data into lowercase.
with open('/content/sentiment_analysis_required_data_set.csv', 'r', encoding = "ISO-8859-1") as file:
  data_file = pd.read_csv (file, error_bad_lines=False)  
#data_file['clean_text']=data_file['clean_text'].str.lower()
#data_file['sentiment']=data_file['sentiment'].str.lower()
data_file

,Document_id,clean_text,sentiment
0,0,I was a person that saw all the hype and claim...,Positive
1,1,Every once in a while a movie comes that trul...,Positive
2,2,This is a movie that only those who have felt ...,Positive
3,3,Truly a masterpiece The Best Hollywood film o...,Positive
4,4,Joaquin Phoenix gives a tour de force performa...,Positive
...,...,...,...
95,95,Dark depressing and unsettling film with a ha...,Positive
96,96,I just didn t get the hype about this movie A...,Negative
97,97,JOKER is a gift to the audiences I felt as a ...,Positive
98,98,Story was just really unconvincing Nobody rea...,Negative


In [50]:
!pip install stop-words
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = []
for i in data_file['clean_text']:
    # clean and tokenize document string
    raw = i.lower()
    if(i=='spoiler content'):
      continue
    tokens.append(tokenizer.tokenize(raw))  

In [67]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokens, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokens], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
def make_bigrams(tokens):
    return [bigram_mod[doc] for doc in tokens]

def make_trigrams(tokens):
    return [trigram_mod[bigram_mod[doc]] for doc in tokens]   

In [0]:
data_words_bigrams = make_bigrams(tokens)
data_words_trigrams=make_trigrams(data_words_bigrams)


In [0]:
id2word = corpora.Dictionary(data_words_trigrams)
corpus = [id2word.doc2bow(t) for t in data_words_trigrams]



In [75]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#1.2 Top 10 Clusters of Topic Modelling
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"the" + 0.014*"to" + 0.014*"and" + 0.014*"damn" + 0.014*"get" + '
  '0.014*"will" + 0.014*"he" + 0.013*"on" + 0.013*"ur" + 0.013*"a"'),
 (1,
  '0.023*"the" + 0.020*"movie" + 0.019*"and" + 0.019*"to" + 0.018*"what" + '
  '0.017*"a" + 0.017*"get" + 0.017*"u" + 0.017*"ur" + 0.017*"damn"'),
 (2,
  '0.039*"movie" + 0.035*"the" + 0.034*"to" + 0.031*"and" + 0.026*"do" + '
  '0.024*"a" + 0.023*"ur" + 0.023*"i" + 0.022*"what" + 0.022*"he"'),
 (3,
  '0.023*"the" + 0.017*"to" + 0.016*"a" + 0.016*"movie" + 0.016*"and" + '
  '0.015*"acting" + 0.015*"really" + 0.015*"u" + 0.015*"joaquin" + '
  '0.015*"will"'),
 (4,
  '0.013*"the" + 0.013*"and" + 0.013*"to" + 0.013*"movie" + 0.013*"damn" + '
  '0.013*"u" + 0.013*"it" + 0.013*"do" + 0.013*"on" + 0.013*"what"'),
 (5,
  '0.023*"the" + 0.016*"movie" + 0.016*"he" + 0.016*"really" + 0.016*"to" + '
  '0.016*"i" + 0.016*"a" + 0.016*"and" + 0.015*"damn" + 0.015*"what"'),
 (6,
  '0.047*"the" + 0.027*"a" + 0.026*"to" + 0.024*"ur" + 0.023*"and" + '

In [80]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts =data_words_trigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.331538741284725

Coherence Score:  0.9999999999999998


In [81]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.010434 -0.004409       1        1  59.201004
2     -0.013029  0.004766       2        1  22.697577
7     -0.015384 -0.000222       3        1  17.795877
6     -0.006725 -0.001109       4        1   0.083723
1      0.002272  0.001688       5        1   0.042212
3      0.004689 -0.000709       6        1   0.040418
5      0.004756 -0.000395       7        1   0.040407
8      0.007775 -0.000267       8        1   0.038049
0      0.012505  0.000215       9        1   0.031634
4      0.013576  0.000441      10        1   0.029105, topic_info=       Term        Freq       Total Category  logprob  loglift
56      the  499.000000  499.000000  Default  30.0000  30.0000
0         a  299.000000  299.000000  Default  29.0000  29.0000
60       to  298.000000  298.000000  Default  28.0000  28.0000
4       and  298.000000  298.000000  Default  27.0000  27.0000
38    movie  297.000000  297.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
1    acting    0.039584  200.694275  Topic10  -4.3175  -0.3891
0         a    0.039584  299.452850  Topic10  -4.3175  -0.7893
33  joaquin    0.039584  200.513718  Topic10  -4.3175  -0.3882
47   really    0.039583  199.871826  Topic10  -4.3175  -0.3850
6      best    0.039582  100.566734  Topic10  -4.3175   0.3018

[515 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.617793       a
0         2  0.183668       a
0         3  0.197026       a
1         1  0.672665  acting
1         2  0.184360  acting
...     ...       ...     ...
73        2  0.249268     you
73        3  0.149561     you
74        1  0.647705    your
74        2  0.159435    your
74        3  0.179365    your

[225 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 3, 8, 7, 2, 4, 6, 9, 1, 5])

Topic 1

# **Question 2: Sentiment Analysis**

(30 points). Sentiment analysis also known as opinion mining is a sub field within Natural Language Processing (NLP) that builds machine learning algorithms to classify a text according to the sentimental polarities of opinions it contains, e.g., positive, negative, neutral. The purpose of this question is to develop a machine learning classifier for sentiment analysis. Based on the dataset from assignment three, write a python program to implement a sentiment classifier and evaluate its performance. Notice: **80% data for training and 20% data for testing**.  

(1) Features used for sentiment classification and explain why you select these features.

(2) Select two of the supervised learning algorithm from scikit-learn library: https://scikit-learn.org/stable/supervised_learning.html#supervised-learning, to build a sentiment classifier respectively. 

(3) Compare the performance over accuracy, precision, recall, and F1 score for the two algorithms you selected. Here is the reference of how to calculate these metrics: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9. 

In [0]:
# Write your code here





# **Question 3: House price prediction**

(40 points). You are required to build a **regression** model to predict the house price with 79 explanatory variables describing (almost) every aspect of residential homes. The purpose of this question is to practice regression analysis, an supervised learning model. The training data, testing data, and data description files can be download here: https://github.com/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/assignment4-question3-data.zip. Here is an axample for implementation: https://towardsdatascience.com/linear-regression-in-python-predict-the-bay-areas-home-price-5c91c8378878. 


In [0]:
# Write your code here